# HIVE Y PIG

## En alto nivel... ¿Qué es HIVE y PIG?
- Herramientas de programación
    - HIVE: nos permite lanzar consultas tipo SQL contra archivos de HDFS
    - PIG: es un nivel de abstracción superior a MapReduce que nos permite ejecutar comandos para el tratamiento avanzado de datos

## Diferencias

|Pig| Hive|
|---|---|
|Lenguaje procedural.| Lenguaje declarativo similar a SQL.|
|Datos des/estructurados|Centrado en datos estructurados.|
|Esquema definido en el programa.|Esquema definido de antemano.|
|Principalmente usado para programar.| Principalmente usado para generar informes.|

# HIVE

## En detalle ... ¿Qué es HIVE?
- Hive es una infraestructura de data warehouse sobre Hadoop que permite la utilización de un cluster de Hadoop sin necesidad de programar MapReduce
- Utiliza un lenguaje de consultas similar a SQL, llamado HiveQL, de forma que desarrolladores que conozcan SQL pueden utilizar Hadoop
- Hive compila las consultas redactadas en HiveQL para crear las tareas MapReduce

## Ventajas
- Es extensible: permite entre otros utilizar tipos de datos y funciones definidos por el usuario, utilizar plugins desarrollados en otros lenguajes
- Es interoperable: Soporta diferentes formatos de datos y de ficheros
- Ofrece altas prestaciones y tolerancia a fallos: Aprovehca las características de Hadoop

## Diferencias entre Hive y BBDD relacionales
|Hadoop con Hive| RDBMS|
|---|---|
|La escalabilidad es su fuerte.| Escalan peor.|
|Diseñado para gestionar consultas de alta latencia.| Diseñados para gestionar consultas SQL de alta y baja latencia.|
|Trabajan sobre petabytes de datos multiestructurados.| Trabajan sobre varios terabytes de datos estructurados.|
|Eficiente en cuanto a costes.| Alto coste de escalado.|

## Arquitectura de HIVE
[03_hadoop_01]:images/03_hadoop_01

- Hiveserver2:
    - Permite conexiones remotas utilizando ODBC/JDBC de forma que se pueden utilizar aplicaciones tales como excel, toad o squirrel
    - Estas conexiones las gestiona el componente Thrift
    - Metastore:
        - Contiene las definiciones de los esquemas de base de datos
        - Es una base de datos relacional, externa a Hive
- Clientes
    - Terminal: beeline
    - Interfaz gráfico web: HUE
    - Herramientas legadas: Toad, SQuirrel ... a través de JDBC/ODBC
- Además, otro componente es HCatalog
    - Permite que otras herramientas (ej. Pig) puedan leery escribir esquemas de datos almacenados en el metastore
    

## Acciones

- Crear Base de datos:
    - Se crea en el metastore
    > CREATE DATABASE
    
- Crear tabla
    - Se crea en el metastore
    > CREATE EXTERNAL TABLE dataexternal( id STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' LOCATION '/path/to/dataexternal';
    
- Selección de datos
    - Consulta al metastore info de esa tabla para elaborar el plan de ejecución
    - Hiveserver crea el plan de ejecución de la consulta (el trabajo MapReduce) y envía al ResManager para ejecutarlo
    - ResManager asigna el AppMaster de este trabajo a un nodo. Además informa al AppMaster de los recursos disponibles.
    - AppMaster:
        - consulta al NM dónde están los ficheros de entrada
        - solicita al ResManager contenedores en los nodemanagers que almacenan los datos
        - contacta con los NodeManagers para crear contenedores
    - Containers 
        - consultan al Namenode dónde están los ficheros de entrada
        - leen datos de entrada y ejecutan los map
    - Los map finalizan. Su salida se almacena en local y se envía a los reduce que comienzan su ejecución
    - Los reduce finalizan. La salida se almacena en HDFS
    - AppManager informa al ResManager de su finalización
    - ResManager informa a Nodemanagers y a Hiveserver del final de la ejecución
    - HiveServer 
        - consulta Namenode para recuperar la salida
        - lee bloques de datanodes
        - resuelve resultados al cliente
    > SELECT COUNT(gsd19024) FROM dataexternal WHERE id = '20'
    
- Modelo de datos
    - Hive soporta una variedad de tipos de datos que se dividen en dos categorías:
        - Primitivos
            - Numéricos, string, fecha/hora, misc.
        - Complejos
            - STRUCT
            - MAP
            - ARRAY

- Tablas
    - En Hive, las tablas son entidades ``virtuales``, ya que los datos se encuentran almacenados en ficheros en HDFS
    - Existen dos tipos de tablas en Hive
        - Tablas gestionadas (managed tables): también llamadas tablas internas
            - Hive controla la tabla por completo. Sus datos están almacenados en un subdirectorio gestionado únicamente por Hive
            - Cuando eliminiamos la tabla, se eliminan también sus datos
        - Tablas externas
            - Los datos están almacenados fuera del control de Hive
            - Cuando se elimina la tabla, solamente se eliminan los metadatos, mientras que los datos persisten
            
- Particiones
    - Definen la forma en que Hive almacena físicamente los datos en disco
    - Permiten agrupar las filas basándose en el valor de alguna columna de interés
    - Es de interés para mejorar la eficiencia de las consultas que requieren escaneos completos de tablas
    - Una partición en Hive es una carpeta en HDFS
    
- Buckets
    - Complementan a las particiones para organizar mejor los datos y de esta forma realizar de forma más eficiente las consultas
    - Al contrario que las particiones, se define un número concreto de buckets, que no se puede modificar tras crear la tabla
    - Los buckets están basados en que items con el mismo valor en una función hash van al mismo bucket
    
|Particiones| Buckets|
|---|---|
|Carpeta en HDFS| Fichero en HDFS|
|Número indefinido de particiones|Número definido de buckets|
|La columna sobre la que realizamos la partición no se incluye en el esquema de la tabla de Hive|La columna sobre la que realizamos el bucket sí se incluye en el esquema de la tabla de Hive|Número de particiones variable|Una vez creada la tabla, no se pueden modificar sus buckets|
|Tamaños de particiones indiferentes|Idealmente, todos los buckets tienen el mismo tamaño.|

- Vistas
    - Las vistas son tablas virtuales que constituyen una ventana hacia la tabla subyacente (conocida como tabla ``base``)
    - Al acceder a una vista, estamos accediendo a la tabla base
    
- Joins
    - Un join se utiliza para recupèrar datos de interés de dos o más tablas basados en un campo común
    - Hive solamente soporta joins de igualdad
    - Poner a la derecha la tabla más grande para realizar un mapjoin si la tabla pequeña cabe en memoria en los mappers
    
- Formatos de serialización y deserialización (SERDE)
    - Serialización y deserialización (o serde) consiste en leer y escribir datos de una variedad de formatos
    - Algunos de los serdes que proporciona Hive por defecto son:
        - LazySimpleSerDe: básico, se usa por defecto
        - RegexSerDe: utiliza expresiones regulares
        - AvroSerDe: utiliza el formato de archivo Avro
        - OrcSerde: utiliza el formato de archivo ORC
        - ParquetHiveSerDe: utiliza el formato de archivo Parquet
        - JSONSerDe: utiliza el formato de archivo JSON
        - CSVSerDe: utiliza el formato de archivo CSV
        
- Planes de ejecución de consultas
    - Calcular estadísticas de las tablas, para el cálculo de los planes de ejecución
    - Orden
        >ANALYZE TABLE dataexternal COMPUTE STATISTICS;
        
- Transacciones
    - Hive proporciona ACID a nivel de fila, de forma que una aplicación puede añadir filas mientras que otra lee de la misma partición sin interferir
    - Limitaciones
        - Transacciones deshabilitadas por defecto
        - BEGIN, COMMIT Y ROLLBACK no están soportadas aún, las opeoraciones del lenguaje son auto-commit
        - Solamente sobre tablas con buckets y en formato ORC
    
- Extensibilidad
    - Hive ofrece la posibilidad de implementar nuevas funcionalidades, mediante funciones definidas por el usuario (User Defined Functions, UDFs) en Java, Python
    
- Compresión
    - Hive permite comprimir datos tanto finales como intermedios

# PIG

## ¿Qué es PIG?

- Pig es una herramienta que permite programar MapReduce sin tener que "pensar" en MapReduce
- Pig permite ejecutar flujos de datos en un cluster paralelo distribuido Apache Hadoop
- Utiliza un lenguaje llamado Pig Latin para definir dichos flujos de datos
- Pig Latin incluye operadores para muchos operadores tradicionales (reunión, ordenación, filtrado, ...)
- Además, Pig Latin permite el desarrollo de funciones ad-hoc para lectura, procesamiento y escritura de datos
- Pig proporciona un enfoque procedural (al contrario que Hive que es declarativo) para el proceso de datos
    - Pig permite la definición de los detalles del procesamiento de los datos, sin tener que confiar en analizadores de consultas que generen un plan de ejcución eficiente.
    
## Comparando Pig Latin con MapReduce

|Pig |MapReduce|
|---|---|
|Lenguaje de alto nivel| Paradigma de bajo nivel|
|Minimiza la complejidad del código (1 consulta Pig puede equivaler a múltiples trabajos MapReduce) y el tiempo de desarrollo|Código aprox. 20 veces más largo y tiempo de desarrollo aprox. 16 veces mayor (es necesario programar cada trabajo individualmente)|
|Proporciona operadores: sort, join, filtros...| Estos operadores se deben implementar manualmente|
|Proporciona tipos de datos anidados,como tuplas o bags|Tipos de datos simples|

## Comparando Pig Lating con SQL

|Pig Latin| SQL|
|---|---|
|Describe el procedimiento a aplicar a unos datos de entrada.|Se centra en responder a una pregunta, pero no describe cómo se debe responder.|
|Se pueden enlazar diversas operaciones en el orden en que se realizan, sin necesidad de almacenar resultados intermedios en tablas temporales.|Se centra en una única pregunta cada vez, y cuando se ecesita más de una operación, es necesario escribir subconsultas separadas o almacenar resultados en tablas intermedias.|

## Estructura de PIG

- Pig Latin
    - Lenguaje de programación en el que se desarrollan los scripts para Pig
- Grunt Shell
    - Shell que permite la ejecución de Pig interactivas
- Pig Server
    - Ejecuta los scripts Pig
- Parser
    - Chequea la sintaxis de los scripts y da como resultado un grafo dirigido acíclico (DAG).
    - El DAG está formado por nodos que son los operadores lógicos y por aristas que son los flujos de datos
- Optimizer
    - Toma el DAG generado por el parser y realisza las modificaciones necsarias para minimizar los datos que están en procesamiento en cada momento
    - Entre otras cosas:
        - Elimina columnas innecesarias
        - Elimina claves map que no se utilizan
- Compiler
    - Genera los trabajos MapReduce para el código optimizado
- Execution engine
    - Se encarga de la ejecución de los trabajos MapReduce
    - Las salidas de estos trabajos se pueden almacenar en HDFS o mostrarse por pantalla
- Tipos de datos primitivos
    - Entre otros: int, long, float, double, chararrary, boolean, datetime, bytearray
    - Map
        - Almacena una clave de tipo chararray y su valor asociado
        - El tipo de datos del valor puede ser uno complejo
        - Si el tipo de datos no se puede determinar, Pig utiliza bnytearray
        - La asociación entre la clave y el valor se define utilizando el símbolo #, y las claves tienen que ser únicas
        - Sintáxis: [key#value, key1#value1,...]
    - Tupla
        - Es una colección de valores de longitud fija y ordenados. Una tupla es similar a un registro en una tabla SQL pero sin restricciones en los tipos de datos de las columnas, y sin tener un esquema asociado
        - Sintáxis: (value1, value2, value3, ...)
    - Bag
        - Es un contenedor de tuplas y otras bags. Están desordenadas, y puede haber tuplas o bas duplicadas dentro de una bag
        - Sintáxis: {(tuple1, tuple2, ...)}
        - Pueden ser internas o externas
            - Interna(inner bag) es una bag dentro de una tupla
            - Externa(outer bag) es una bag de tuplas
    - Pig permite el anidamiento de estructuras de datos complejas donde se puede anidar una tupla dentro de otra tupla, una bag y un map
    - Null: cualquier tipo de datos puede valuer Null
        - El dato es desconocido, no existe, un error
    - Para comprender mejor estos conceptos
        - Una relación (una tabla de SQL) es una bag
        - Una bag es una colección de tuplas
        - Una tupla (similar a una fila de una base de datos) es un conjunto ordenado de campos
        - Un campo es un dato
- Esquemas
    - Pig puede trabajar con esquemas, pero no impide trabajar sin ellos
        - En el caso de que definamos un esquema, Pig lo aplicará a los datos que le pasemos, tanto para chequear que se cumple como para optimizadores
        - En el caso de que no definamos un esquema, Pig tratará dichos datos de la mejor forma que pueda
    - Además, los esquemas se pueden definir con la granularidad que queramos
        - Ejemplo: un esquema puede contener tuplas, pero no es necesarios indicar la estructura de la tupla
    - Pig trata de inferir los tipos de datos cuando no tiene información
    - Se considera que Pig no es fuertemente tipado

- Cargar esquemas
    - La forma más sencilla de cargar un esquema es indicarlo cuando cargamos los datos
        - Orden
            > LOAD 'data' [USING function][AS Schema];
        - Ejemplo
            > relation = load'/path/to/file'; -- Carga un fichero de datos separados por tabuladores, utilizando la función de carga por defecto PigStorage
            >
            > relation = load'/path/to/file' using PigStorage(); -- Carga un fichero de datos separados por comas
            >
            > relation = load'/path/to/file' as (campo1,campo2) -- Carga un fichero de datos y especifica un esquema
    - Esta forma es tediosa para esquemas con muchas columnas y para esquemas cambiantes
    - Otra forma de especificar dónde se encuentra el esquema para los datos que se cargan
    - Se puede utilizar un esquema creado previamente en Hive
        - Ejemplo
            - mdata = load'database.tabla' using org.apache.hive.hcatalog.pig.HCatLoader();

- Operaciones fundamentales sobre datos
    - Los operadores de Pig se pueden agrupar en las siguientes familias
        - Operaciones de control, conectables y estructurales
        - Operadores de control
            - Operadores de serialización: LOAD y STORE
            - Directivas de Pig (DESCRIBE, ILLUSTRATE, REGISTER,...) que sriven para que Pig muestre información, registrar funciones, ...
            
- Almacenamiento de datos
    - Almancenar datos
        - Orden
        > STORE alias INTO 'directory' [USING function];
        - Ejemplos
        > STORE procesados INTO 'path/to/file';
            - Almacena los datos procesados separados por tabuladores en la carpeta indicada en HDFS utilizando la función de almacenamiento por defecto PigStorage
        > STORE procesados INTO 'path/to/file' using PigStorage(',');
            - Similar al anterior pero utilizando el separador ','
        - Podemos escribir en una tabla existente en Hive
        > STORE angio3 INTO 'angio3' USING org.apache.hive.hcatalog.pig.HCatStorer()
    - Mostrar por pantalla
        - Orden
        > DUMP alias
        - Ejemplo
        > DUMP tablefile

- Operaciones fundamentales sobre datos
    - Operadores conectables
        - Sirven para crear un trabajo solo map
        - Cuando van antes o después de una operación estructural, van al map o al reducer
        
        - Operaciones de transformación:
            - FOREACH, FOREACH..FLATTEN(tuple) modifican los contenidos de los registros de forma individual
            - La cuenta de los registros de salida es la misma que la de los registros de entrada, pero se puede modificar su contenido o su esquema
        - Operaciones de filtrado
            - FILTER, SAMPLE, LIMIT, ASSERT aceptan o rechazan cada registro de forma individual
            - Pueden producir el mismo número de registros de entrada o menos, pero cada registro tiene el mismo contenido y esquema que a la entrada
        - Operadores de repartición
            - SPLIT, UNION no cambian los registros, solamente los distribuyen en nuevas trablas o flujos de datos
            - UNION produe tantos registros como la suma de su entrada
            - SPLIT sin varios FIlTER simultaneos, los registros de salida que produce son la suma de lo que produce cada uno de sus filters
            - Operaciones de desagrupado
                - FOREACH..FLATTEN(bag) convierten registros que son bags de tupl,as en registros que contienen cada uno una combinación de las tuplas
                - FLATTEN deja los contenidos de la bag como estaban y sustituyue el esquema de la bag por el esquema de sus contenidos. Cuando aplicamos esta función a un único campo, la cuenta de registros de salida es la misma que la cuenta de los elementos de todas las bags
                - Múltiples FLATTEN generan un registro por cada combinación posible de elementos, que puede ser mucho más grande que la entrada
    - Operaciones estructurales
      - Estas operaciones requieren un map y un reduce
      - Operaciones de agrupación (GROUP, COGROUP, CUBE, ROLLUP):
          - Colocan los registros en contexto con respecto a los demás. No modifican los contenidos de los registros pero sí que recolocal su esquema
              - Se les encuentra seguidos de FOREACH que aprovecha la agrupación generada
              - GROUP y COGROUP emiten un registro por cada valor de agrupación
      - Operaciones JOIN, CROSS
          - Emparejan registros entre tablas    
          - JOIN es una secuencia COGROUP/FLATTEN/FOREACH/ optimizada
      - Operaciones de ordenación (ORDER BY, RANK)
          - Ordenan su entrada: Cada registro en el fichero 00000 se coloca en orden y va antes que cualquier registro del fichero 00001... y así para todos los ficheros de entrada
          - Esto requiere dos trabajos, primero un map para comprender la distribución de las claves utilizadas para la ordenación, y luego un MapReduce para realizar la ordenación
      - Operaciones de unicidad (DISTINCT, COGROUP)
          - Seleccionan/rechazan/eliminan duplicados
          - Esto se consigue con combinaciones de los de arriba, y puede involucrar más de un trabajo MapReduce
- Directivas de control
    - Mostrar el esquema de una relación
        > DESCRIBE relationship
    - Mostrar un subconjunto de los datos de una relación
        > SAMPLE relationship <sampling:number>
        - Se suele utilizar en conjunción con limit y dump
        > sampled = SAMPLE relationship 0.10
        >
        > limited = LIMIT sampled 5;
        > 
        > DUMP limited
    - Mostrar cómo se crea una relación
        - Simula la ejecución de una relación para un subconjunto de los datos aleatorio
            > ILLUSTRATE relationship
    - Mostrar el plan de ejecución de un comando
        > EXPLAIN relationship
        
- Operadores relacionales
    - Genera transformaciones basadas en columnas de datos
        > alias = FOREACH{gen_blk|nested_gen_blk}GENERATE expression;
        >
        > comienzo = FOREACH relationship GENERALTE ..field2
    - Genera transformaciones basadas en filas de datos
        > alias = FILTER alias BY expression;
        >
        > bb = filter test2 by pip23>100 and pin23<400;
    - Agrupa datos en una o varias relaciones
        > alias = GROUP alias {ALL|BY expression}[,alias ALL|BY expression ...][USING 'collected'][PARALLEL n];
        >
        > alias = COGROUP alias {ALL|BY expression}[,alias ALL|BY expression ...][USING 'collected'][PARALLEL n]
        >
        > bb = group test4 by pinpin;
    - Ordenar resultados
        > bb = order test4 by pinpin123 desc;
    - INNER
        > Z = JOIN X BY x1, Y BY y1
    - OUTER
        > Z = JOIN X BY $0 FULL, Y BY $0
    - LIMIT
        > Z = LIMIT X 10
    - Paralelismo
        > parallelism = group day by character parallel 5;
- Funciones en Pig
    - Matemáticas: log, exp, ...
    - Comparación de cadenas: ==, matches ...
    - Transformar cadenas: CONCAT, LOWER, ...
    - Fecha y hora: CurrentTime, ToUnixTime, SecondsBetween,...
    - Funciones de agregación sobre bags: AVG, MAX, MIN ....
    - Funciones sobre bags: TOP, SUBSTRACT
- UDF a través de Piggybank y Apache DataFu
    - Son librerías que se distribuyen con Pig y se pueden utilizar registrando su jar con REGISTER
    - Para utilizar una UDF, debemos usar su path completo, aunque podemos crear atajos mediante DEFINE
    - Ejemplo
        - Register /usr/lib/pig/piggiybank.jar
        - DEFINE Reverse org.apache.pig.piggybank.evaluation.string.Reverser()
        - b = FOREACH a GENERATE Reverse(char_field) AS reversed_char_field;